Ce notebook de travail est utilisé pour concevoir le temps de parcours réel en utilisant les données du topic Kafka, faire une anti-jointure, et le rebalancer dans une nouvelle topic kafka.

In [1]:
from kafka import KafkaConsumer, TopicPartition
from kafka.consumer.fetcher import log
import json
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.apache.kafka:kafka-clients:3.2.0'

def get_end_offsets(consumer, topic) -> dict:
    partitions_for_topic = consumer.partitions_for_topic(topic)
    if partitions_for_topic:
        partitions = []
        for partition in consumer.partitions_for_topic(topic):
            partitions.append(TopicPartition(topic, partition))
        # https://kafka-python.readthedocs.io/en/master/apidoc/KafkaConsumer.html#kafka.KafkaConsumer.end_offsets
        # Get the last offset for the given partitions. The last offset of a partition is the offset of the upcoming message, i.e. the offset of the last available message + 1.
        end_offsets = consumer.end_offsets(partitions)
        return end_offsets

In [2]:
def grab_topic_gare(gare: str = "87271460", num : int = 2):
    last_n_msg = num
    # kafka_server = "35.180.29.24:9092"
    kafka_server = "13.37.146.224:9092"
    # consumer
    consumer = KafkaConsumer(
        bootstrap_servers=kafka_server,
        consumer_timeout_ms=10000)
    end_offsets = get_end_offsets(consumer, 'rer-b-'+gare)
    consumer.assign([*end_offsets])
    for key_partition, value_end_offset in end_offsets.items():
        new_calculated_offset = value_end_offset - last_n_msg
        new_offset = new_calculated_offset if new_calculated_offset >= 0 else 0
        consumer.seek(key_partition, new_offset)

    for msg in consumer:
        return msg

In [3]:
def forgiving_json_deserializer(v):
    if v is None:
        try:
            return json.loads(v.encode('utf-8'))
        except json.decoder.JSONDecodeError:
            log.exception('Unable to decode: %s', v)
        return None

In [4]:
content_1 = grab_topic_gare("87271411", 1)
content_2 = grab_topic_gare("87271411", 2)

In [5]:
import pyspark
from pyspark.sql import *

In [6]:
from pyspark.shell import sc

spark = SparkSession.builder\
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()


22/06/27 19:29:17 WARN Utils: Your hostname, Rakibs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.9 instead (on interface en0)
22/06/27 19:29:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/27 19:29:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.10.4 (main, Mar 24 2022 17:42:03)
Spark context Web UI available at http://10.0.0.9:4040
Spark context available as 'sc' (master = local[*], app id = local-1656350958802).
SparkSession available as 'spark'.
22/06/27 19:29:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
df_previous = spark.read.json(sc.parallelize([json.loads(content_1[6])]))
df_latest = spark.read.json(sc.parallelize([json.loads(content_2[6])]))

In [8]:
df_latest.show()

+----------------+---------+--------+----+----+------+--------+-------------------+
|            date|direction|    etat|miss|mode|   num|    term|      time_recorded|
+----------------+---------+--------+----+----+------+--------+-------------------+
|27/06/2022 19:28|        0|        |LOVE|   R|LOVE44|87758847|2022-06-27 19:28:01|
|27/06/2022 19:29|        0|        |IRMA|   R|IRMA09|87271528|2022-06-27 19:28:01|
|27/06/2022 19:31|        1|        |EMUE|   R|EMUE11|87001479|2022-06-27 19:28:01|
|27/06/2022 19:34|        1|        |IJEL|   R|IJEL13|87271528|2022-06-27 19:28:01|
|27/06/2022 19:34|        0|        |STOP|   R|STOP48|87758722|2022-06-27 19:28:01|
|27/06/2022 19:37|        1|        |PPIT|   R|PPIT50|87758896|2022-06-27 19:28:01|
|27/06/2022 19:41|        1|        |PUMA|   R|PUMA52|87758896|2022-06-27 19:28:01|
|27/06/2022 19:43|        1|        |IRMA|   R|IRMA17|87271528|2022-06-27 19:28:01|
|27/06/2022 19:46|        1|        |STOP|   R|STOP54|87758722|2022-06-27 19

In [9]:
df_previous.show()

+----------------+---------+--------+----+----+------+--------+-------------------+
|            date|direction|    etat|miss|mode|   num|    term|      time_recorded|
+----------------+---------+--------+----+----+------+--------+-------------------+
|27/06/2022 19:31|        1|        |EMUE|   R|EMUE11|87001479|2022-06-27 19:29:02|
|27/06/2022 19:34|        0|        |STOP|   R|STOP48|87758722|2022-06-27 19:29:02|
|27/06/2022 19:35|        1|        |IJEL|   R|IJEL13|87271528|2022-06-27 19:29:02|
|27/06/2022 19:37|        1|        |PPIT|   R|PPIT50|87758896|2022-06-27 19:29:02|
|27/06/2022 19:40|        1|        |PUMA|   R|PUMA52|87758896|2022-06-27 19:29:02|
|27/06/2022 19:43|        1|        |IRMA|   R|IRMA17|87271528|2022-06-27 19:29:02|
|27/06/2022 19:46|        1|        |STOP|   R|STOP54|87758722|2022-06-27 19:29:02|
|27/06/2022 19:49|        1|        |KFON|   R|KFON56|87393579|2022-06-27 19:29:02|
|27/06/2022 19:53|        1|        |EMUE|   R|EMUE19|87001479|2022-06-27 19

In [10]:
df_final = df_latest.join(df_previous, df_latest.num == df_previous.num, 'left_anti')

In [11]:
df_final.show()

+----------------+---------+----+----+----+------+--------+-------------------+
|            date|direction|etat|miss|mode|   num|    term|      time_recorded|
+----------------+---------+----+----+----+------+--------+-------------------+
|27/06/2022 19:29|        0|    |IRMA|   R|IRMA09|87271528|2022-06-27 19:28:01|
|27/06/2022 19:28|        0|    |LOVE|   R|LOVE44|87758847|2022-06-27 19:28:01|
+----------------+---------+----+----+----+------+--------+-------------------+



In [12]:
from pyspark.sql.functions import lit

df_final.show()

+----------------+---------+----+----+----+------+--------+-------------------+
|            date|direction|etat|miss|mode|   num|    term|      time_recorded|
+----------------+---------+----+----+----+------+--------+-------------------+
|27/06/2022 19:29|        0|    |IRMA|   R|IRMA09|87271528|2022-06-27 19:28:01|
|27/06/2022 19:28|        0|    |LOVE|   R|LOVE44|87758847|2022-06-27 19:28:01|
+----------------+---------+----+----+----+------+--------+-------------------+



In [13]:
df_final = df_final\
    .withColumn("gare", lit(87271411)) \
    .withColumn("time_arrived", lit(df_previous.collect()[0][-1]))

In [19]:
pandas_df = df_final.select("gare", "date", "mode", "num", "miss", "direction", "term", "time_recorded", "time_arrived", "etat").toPandas()

In [20]:
pandas_df

,gare,date,mode,num,miss,direction,term,time_recorded,time_arrived,etat
0,87271411,27/06/2022 19:29,R,IRMA09,IRMA,0,87271528,2022-06-27 19:28:01,2022-06-27 19:29:02,
1,87271411,27/06/2022 19:28,R,LOVE44,LOVE,0,87758847,2022-06-27 19:28:01,2022-06-27 19:29:02,


In [21]:
pandas_df.to_csv("test.csv", index=False)

In [27]:
from kafka import KafkaProducer


def send_to_kafka(data: dict):
    """
    Méthode qui va envoyer vers un kafka le résultat du prétraitement pour pouvoir
    être consommé par les applications
    """
    topic = 'rer-b-computed'
    producer = KafkaProducer(bootstrap_servers="localhost:9092")
    producer.send(topic, value=(json.dumps(data, ensure_ascii=False).encode('utf-8')))
    producer.flush()
    print("All data sent to kafka")

In [23]:
di = pandas_df.to_dict(orient="records")

In [14]:
#ds = df_final.selectExpr("CAST(gare AS STRING)", "to_json(struct(*)) AS value") \
#    .write \
#    .format("kafka") \
#    .option("kafka.bootstrap.servers", "localhost:9092") \
#    .option("kafka.security.protocol", "PLAINTEXT") \
#    .option("topic", "RER-B_Stream") \
#    .save()

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        

In [25]:
json.dumps(di).encode(encoding="UTF-8")

b'[{"gare": 87271411, "date": "27/06/2022 19:29", "mode": "R", "num": "IRMA09", "miss": "IRMA", "direction": 0, "term": 87271528, "time_recorded": "2022-06-27 19:28:01", "time_arrived": "2022-06-27 19:29:02", "etat": ""}, {"gare": 87271411, "date": "27/06/2022 19:28", "mode": "R", "num": "LOVE44", "miss": "LOVE", "direction": 0, "term": 87758847, "time_recorded": "2022-06-27 19:28:01", "time_arrived": "2022-06-27 19:29:02", "etat": ""}]'

In [28]:
send_to_kafka(di)

All data sent to kafka
